```
Notes:
Need to do the whole shebang with all the images
use color and resize with the ImageDataGenerator from keras on the fly
Will give us that extra dimension we need. 
```

## Review

Our last classsifier was very poor--it operated at chance--a coin flip would have had the same predictive power. A few things may have been going on that casued us to find no signal. I could be we down sampled our images too much and lost useful information, it could be that our model was poorly configured (it was), it could be we were using thr wrong model (we were), or it could be all of these. My gut (and eyes) say there is still plenty of information in our down sampled images, so we're going to try a more approproate model before we go back and try with larger images. 

We're going to build a convolutional neural network using keras (with Tensorflow as the backend). For starters, convolutional neural networks are much better at image classification than the mutli-layer perceptron used in the last post. 

In [14]:
# keeps warnings from printing for nicer blog output
import warnings 
warnings.filterwarnings('ignore') 

import os

import pickle # for loading our extracted features
import pandas as pd # for DataFame class our features are loaded into to
import numpy as np

from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, AveragePooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import CSVLogger

csv_logger = CSVLogger('log.csv', append=True, separator=';')

# for processing images
from skimage.transform import rescale 

In [10]:
os.chdir(os.path.expanduser('~/share/rkingdc-blog/regplot'))

We can see this is the same file we saw in the previous post, but with the image data as an added column. Each cell in that `img_series` column is a numpy ndarray of shape `(168,168)`. 

## Building the Model

We'll start by building our classifier. Code will be explained inline. 

In [19]:
mod = Sequential()

mod.add(Conv2D(filters=12, 
               kernel_size=(4,4), 
               strides=(2,2), 
               input_shape=(256,256,3),
               activation='relu'))

# pooling layer reduces model complexity--good to run before flattening 
mod.add(MaxPooling2D(pool_size = (2,2)))

mod.add(Conv2D(filters=8, 
               kernel_size=(4,4), 
               strides=(2,2), 
               input_shape=(128,128,3), # we've got 240x240 px photos in color
               activation='relu'))

mod.add(AveragePooling2D(pool_size = (2,2)))

mod.add(Flatten())

# our hidden layer
mod.add(Dense(units=60, activation='relu'))
mod.add(Dense(units=50, activation='relu'))
# our output layer
# binary classification (for now) so we just need one layer 
# sigmoid forces 0,1 output
mod.add(Dense(units=1, activation='sigmoid'))
mod.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
mod.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_13 (Conv2D)           (None, 239, 239, 12)      588       
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 119, 119, 12)      0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 58, 58, 8)         1544      
_________________________________________________________________
average_pooling2d_5 (Average (None, 19, 19, 8)         0         
_________________________________________________________________
flatten_6 (Flatten)          (None, 2888)              0         
_________________________________________________________________
dense_15 (Dense)             (None, 60)                173340    
_________________________________________________________________
dense_16 (Dense)             (None, 50)                3050      
__________

## Pre-procesing Data

In image processing we'll want to pre-proccess our images before we train a model on them, by adding some random stretching, blurring, rotating, etc. Keras has utilities included to make this easier. 

In [28]:
train_gen = ImageDataGenerator()
test_gen = ImageDataGenerator()

In [29]:
train = train_gen.flow_from_directory('data/imgs/train',
                          #   target_size=(240,240),
                             class_mode='binary')
test = test_gen.flow_from_directory('data/imgs/test',
                        #     target_size=(240,240),
                            class_mode='binary')

Found 16000 images belonging to 2 classes.
Found 4000 images belonging to 2 classes.


In [8]:
mod.fit_generator(train,
       epochs=30,
       verbose=0,
       validation_data=test,
       callbacks=[csv_logger])

In [9]:
from datetime import date
mod.save(f'trained_model_1_{str(date.today())}.h5')

In [11]:
model_eval = mod.evaluate_generator(test, use_multiprocessing=True, workers=2)
print(mod.metrics_names)
print(model_eval)